In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# datasetname='TabFact_potable'
datasetname='tabfact_cot'
# datasetname='WikiTableQuestions'

In [3]:
import pandas as pd
import openai
import os
import json
from tqdm import tqdm
from tabqa.GptPrompter import *
from tabqa.GptCOTPrompter import *
from tabqa.GptCOTPrompter_BeamSeach import *
from tabqa.AutoReasoner import *
import dotenv

config = dotenv.dotenv_values("../.env")
openai.api_key = config['OPENAI_API_KEY']
openai.base_url = config['GPT_BASE_URL']

dataset = pd.read_csv('../dataset/'+str(datasetname)+'/data/test.jsonl', sep='\t')

In [4]:
from dataset.tabfact_cot.load_data import load_tabfact_dataset
dataset_path='../dataset/'+str(datasetname)+'/data/test.jsonl'
raw2clean_path='../dataset/'+str(datasetname)+'/data/raw2clean.jsonl'
first_n=10
dataset = load_tabfact_dataset(dataset_path, raw2clean_path, first_n=first_n)
dataset = pd.DataFrame(dataset)

Loading tabfact-test dataset: 100%|██████████| 10/10 [00:00<00:00, 11137.29it/s]


In [11]:
aa = pd.read_csv('../dataset/WikiTableQuestions/csv/200-csv/47.csv', on_bad_lines='skip', sep=",")
print(aa)
print(type(aa))
# 提取列名
columns = x[0]
# 转换为 DataFrame
y = pd.DataFrame(x[1:], columns=columns)
# y= pd.DataFrame(x)
print(type(y))
print(y)

               Date              Event Duration\n(months) Duration\n(years)
0              1927  Experiment set up                NaN               NaN
1              1930   The stem was cut                NaN               NaN
2     December 1938      1st drop fell             96–107           8.0–8.9
3     February 1947      2nd drop fell                 99               8.2
4        April 1954      3rd drop fell                 86               7.2
5          May 1962      4th drop fell                 97               8.1
6       August 1970      5th drop fell                 99               8.3
7        April 1979      6th drop fell                104               8.7
8         July 1988      7th drop fell                111               9.2
9  28 November 2000      8th drop fell                148              12.3
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
  game     date        opponent result wildcats points opponents      record
0    1  sep

In [16]:
NNDemo = False
max_demo = 8
# gpt_model = 'gpt-4o-mini'
gpt_model = 'Qwen/Qwen2.5-7B-Instruct'

program = 'sql-py'
template = 'original-sql-py-no-intermediate'

def parallel_codex_func_formatv1(i):
    max_retry = 3
    while max_retry>0:
        try:
            codex_prompter = CodexAnswerCOTExecutor_HighTemperaturMajorityVote(
                                                f'prompt_template/{template}.json',
                                                dataset.iloc[i]['id'], 
                                                dataset.iloc[i]['statement '], 
                                                dataset.iloc[i]['table_text'], 
                                                dataset.iloc[i]['label'],  
                                                base_path=f'../dataset/{datasetname}/',
                                                demo_file=f'few-shot-demo/WikiTQ-{program}.json',
                                                )
            codex_prompter.max_demo = max_demo
            codex_prompter.model = gpt_model
            codex_prompter._gen_gpt_prompt(NNDemo) #使用 table_formater 函数格式化 source_table_df 
            codex_prompter._get_gpt_prediction_majority_vote(repeat_times=5)
            log = codex_prompter._log_dict()
            break
        except Exception as e:
            log = {
                'id': dataset.iloc[i]['id'],
                'uncaught_err': str(e)
            }
            if "model's maximum context length" in str(e):
                return log
            max_retry -= 1
    print("YES")
    return log
    
n_threads = 3
maxLimit = 8

from joblib import Parallel, delayed
output_result_file = f'../dataset/{datasetname}/results/CodexAnswerCOTExecutor_HighTemperaturMajorityVote_{template}_{program}_NNDemo={NNDemo}_results_pristine-unseen-tables_limit{maxLimit}_model{gpt_model.split("/")[-1]}.json'
output_result_file = f'../dataset/{datasetname}/results/ipy_1.json'

output_dir = os.path.dirname(output_result_file)  # 获取文件的目录部分
if not os.path.exists(output_dir):  # 如果目录不存在
    os.makedirs(output_dir)  # 创建目录
logs = Parallel(n_jobs=n_threads, require='sharedmem')(delayed(parallel_codex_func_formatv1)(i) for i in tqdm(range(min(maxLimit, dataset.shape[0]))))    
json.dump(logs, open(output_result_file, 'w'), indent=4)
print("1")

100%|██████████| 8/8 [00:00<00:00, 1007.37it/s]

YES
YES
YES
YES
YES
YES
YES
YES
1


In [17]:
logs

[{'id': 'test-0', 'uncaught_err': "'statement '"},
 {'id': 'test-1', 'uncaught_err': "'statement '"},
 {'id': 'test-2', 'uncaught_err': "'statement '"},
 {'id': 'test-3', 'uncaught_err': "'statement '"},
 {'id': 'test-4', 'uncaught_err': "'statement '"},
 {'id': 'test-5', 'uncaught_err': "'statement '"},
 {'id': 'test-6', 'uncaught_err': "'statement '"},
 {'id': 'test-7', 'uncaught_err': "'statement '"}]

In [15]:
# evaluate: 
os.system(f'cd ../dataset/{datasetname}/ && python2 evaluate.py ./results/{output_result_file.split("/")[-1]} ')

  File "evaluate.py", line 61
    print(f"Error")
                 ^
SyntaxError: invalid syntax


256